# JSON-RPC (JavaScript Object Notation Remote Procedure Call

hints for installing and using [json-rpc-util](https://github.com/vtss/json-rpc-util):
- install ruby: https://wiki.archlinux.org/title/Ruby
- install dependency 'net-http-digest_auth': https://www.rubydoc.info/gems/net-http-digest_auth/1.1.1/Net/HTTP/DigestAuth
    - `gem install net-http-digest_auth`
- get overview of json spec also from web interface by calling:
`url/json_spec`

In [1]:
from requests import post
from requests.auth import HTTPBasicAuth

In [2]:
# JSON_ENDPOINT = "http://127.0.0.1:5555/json_rpc"
JSON_ENDPOINT = "http://192.168.1.201/json_rpc"
USER = "tsn"
PASSWORD = "tsn"

In [3]:
def get_auth(user, password):
    return HTTPBasicAuth(user, password)

In [4]:
def send_rpc(url, auth, payload):
    re = post(url, auth=auth, json=payload)
    re = re.json()
    if re["error"] is not None:
        print("\nJSON RPC Response Error")
        print("Payload sent:", payload)
        print("Response received:", re)

In [5]:
def get_port_id(port):
    if 0 < port < 7:
        port_id = "Gi 1/{}".format(port)
    elif port == 7:
        port_id = "2.5G 1/1"
    elif port == 8:
        port_id = "2.5G 1/2"
    else:
        print("Error, unknown port", port)
    return port_id

In [6]:
def get_payload_change_tas_state(port, state):
    port_id = get_port_id(port)

    return {
        "method": "tsn.config.interface.tas.params.set",
        "params": [port_id, {"GateEnabled": state}],
        "jsonrpc": "2.0",
        "id": 0,
        }

In [7]:
def get_payload_change_cts_state(port, queue, state):
    port_id = get_port_id(port)

    return {
        "method": "qos.config.interface.scheduler.set",
        "params": [port_id, queue, {'CutThrough': state}],
        "jsonrpc": "2.0",
        "id": 0,
        }

In [8]:
def get_payload_change_clock_sync_interval(clock_id, port, sync_interval):
    port_id = get_port_id(port)

    return {
        "method": "ptp.config.clocks.portDs.set",
        "params": [clock_id, port_id, {'logSyncInterval': sync_interval}],
        "jsonrpc": "2.0",
        "id": 0,
        }

In [9]:
basic_auth = get_auth(USER, PASSWORD)

In [156]:
tas_state = False

for port in range(1, 9):
    payload = get_payload_change_tas_state(port, tas_state)
    send_rpc(JSON_ENDPOINT, basic_auth, payload)

In [176]:
cts_state = False

for port in range(1, 9):
    for queue in range(0, 8):
        payload = get_payload_change_cts_state(port, queue, cts_state)
        send_rpc(JSON_ENDPOINT, basic_auth, payload)

In [191]:
clock_id = 0
sync_interval = -5

for port in range(1, 5):
    payload = get_payload_change_clock_sync_interval(clock_id, port, sync_interval)
    send_rpc(JSON_ENDPOINT, basic_auth, payload)

In [19]:
payload_get_qos = {
        "method": "qos.config.interface.scheduler.get",
        "params": [],
        "jsonrpc": "2.0",
        "id": 0,
        }

payload_get_tsn = {
        "method": "tsn.config.interface.fp.get",
        "params": [],
        "jsonrpc": "2.0",
        "id": 0,
        }

payload_set_qos = {
        "method": "qos.config.interface.scheduler.set",
        "params": ['Gi 1/1', 0, {'CutThrough': True, 'FramePreemption': True}],
        "jsonrpc": "2.0",
        "id": 0,
        }

test = {'method': 'ptp.config.clocks.portDs.set', 'params': [0, 'Gi 1/1', {'logSyncInterval': -5}], 'jsonrpc': '2.0', 'id': 0}

test2 = {
        "method": "ptp.config.clocks.portDs.get",
        "params": [],
        "jsonrpc": "2.0",
        "id": 0,
        }

re = post(JSON_ENDPOINT, auth=basic_auth, json=test2)
re = re.json()
re

{'id': 0,
 'error': None,
 'result': [{'key': [0, '10G 1/1'],
   'val': {'enabled': 1,
    'logAnnounceInterval': -3,
    'announceReceiptTimeout': 3,
    'logSyncInterval': -1,
    'delayMechanism': 2,
    'logMinPdelayReqInterval': 0,
    'delayAsymmetry': 0,
    'ingressLatency': 0,
    'egressLatency': 0,
    'portInternal': False,
    'versionNumber': 2,
    'mcastAddr': 'default',
    'notSlave': False,
    'localPriority': 128,
    'c8021asNeighborPropDelayThresh': 9223372036854775807,
    'c8021asSyncReceiptTimeout': 3,
    'c8021asAllowedLostResponses': 3,
    'c8021asAllowedFaults': 9,
    'c8021asUseMgtSettableLogAnnounceIntvl': False,
    'c8021asUseMgtSettableLogSyncIntvl': False,
    'c8021asUseMgtSettableLogPdelayReqIntvl': False,
    'c8021asMgtSettableLogAnnounceIntvl': 0,
    'c8021asMgtSettableLogSyncIntvl': -3,
    'c8021asMgtSettableLogPdelayReqIntvl': 0,
    'c8021asUseMgtSettableLogGptpCapMsgIntvl': False,
    'c8021asMgtSettableLogGptpCapMsgIntvl': 0,
    'c8021